In [1]:
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import praw
from praw.models import Submission
from psaw import PushshiftAPI
from prawcore.exceptions import Forbidden,NotFound
import csv
import os
from urllib.error import HTTPError
import glob
import requests
import json
from json import JSONDecodeError
import datetime
import pickle
from collections import defaultdict

/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import re

_RE_COMBINE_WHITESPACE = re.compile(r"\s+")

def strip_whitespace(text):
    return _RE_COMBINE_WHITESPACE.sub(" ", text).strip()

In [3]:
def get_submissions(reddit_instance,subreddit_str):
    
    if not os.path.exists('praw_output'):
        os.mkdir('praw_output')
    
    subreddit = reddit_instance.subreddit(subreddit_str)
    print('Getting submissions and comments from: {}'.format(subreddit.display_name))  
    try:
        title = subreddit.title
        desc = subreddit.description

        if not os.path.exists('subreddits.tsv'):
            with open('subreddits.tsv','w') as f:
                csvwriter = csv.writer(f, delimiter='\t')
                csvwriter.writerow([subreddit.display_name,title,desc])
        else:
            with open('subreddits.tsv','a') as f:
                csvwriter = csv.writer(f, delimiter='\t')
                csvwriter.writerow([subreddit.display_name,title,desc])

        # Write header
        with open(os.path.join('praw_output','{}.tsv'.format(subreddit.display_name)), 'w', newline='\n') as csvfile:
            csvwriter = csv.writer(csvfile, delimiter='\t',
                                    quotechar='|', quoting=csv.QUOTE_MINIMAL)
            csvwriter.writerow(['title','author','date','is_video','id','num_downs','num_ups','upvote_ratio',
                               'num_comments','score','text','subreddit'])

        # Write body
        for submission in subreddit.new(limit=None):
            sub_title = submission.title
            sub_author = submission.author.name if submission.author is not None else -1
            sub_date = submission.created
            sub_is_vid = submission.is_video
            sub_id = submission.id
            sub_downvotes = submission.downs
            sub_upvotes = submission.ups
            sub_upvote_ratio = submission.upvote_ratio
            sub_num_comments = submission.num_comments
            sub_score = submission.score
            sub_text = submission.selftext.strip().replace('\t','').replace('\n','')
            sub_subreddit = submission.subreddit.display_name
            with open(os.path.join('praw_output','{}.tsv'.format(subreddit.display_name)), 'a', newline='\n') as csvfile:
                csvwriter = csv.writer(csvfile, delimiter='\t',
                                    quotechar='|', quoting=csv.QUOTE_MINIMAL)
                csvwriter.writerow([sub_title,sub_author,sub_date,sub_is_vid,sub_id,sub_downvotes,
                                   sub_upvotes,sub_upvote_ratio,sub_num_comments,sub_score,sub_text,sub_subreddit])
    except HTTPError as e:
        if e.code == 403:
            print('Forbidden: private subreddit.')
            
def get_submission_comments(reddit_instance,subreddit,submission_id):
    
    submission = Submission(reddit_instance,id=submission_id)
    
    try:
        submission.comments.replace_more(limit=0)
        all_comments = submission.comments.list()

        # Write header
        with open(os.path.join('praw_output','post_comments','{}_COMMENTS.tsv'.format(subreddit)), 'w', newline='\n') as csvfile:
            csvwriter = csv.writer(csvfile, delimiter='\t',
                                    quotechar='|', quoting=csv.QUOTE_MINIMAL)
            csvwriter.writerow(['submission_id','author','text','date','id','controversiality','num_downs','num_ups',
                               'num_likes','score','subreddit'])

        # Write body
        for comment in all_comments:
            sub_id = comment._submission.id
            assert sub_id == submission_id
            author_name = comment.author.name if comment.author is not None else -1
            comment_body = comment.body.strip().replace('\t','').replace('\n','')
            date_created = comment.created
            comment_id = comment.id
            controversiality = comment.controversiality
            num_downs = comment.downs
            num_ups = comment.ups
            num_likes = comment.likes
            score = comment.score
            subreddit_name = comment.subreddit.display_name
            #print(subreddit_name,subreddit)
            assert subreddit_name == subreddit
            
            with open(os.path.join('praw_output','post_comments','{}_COMMENTS.tsv'.format(subreddit_name)), 'a', newline='\n') as csvfile:
                csvwriter = csv.writer(csvfile, delimiter='\t',
                                    quotechar='|', quoting=csv.QUOTE_MINIMAL)
                csvwriter.writerow([sub_id,author_name,comment_body,date_created,comment_id,controversiality,
                                   num_downs,num_ups,num_likes,score,subreddit_name])
    except HTTPError as e:
        if e.code == 403:
            print('Forbidden: private subreddit.')

In [16]:
with open('../keywords_long.txt','r') as f:
    KEYWORDS_LONG = f.read().splitlines()
KEYWORDS_SHORT = set(["climate change","global warming","carbon","co2","methane",
                  "green","environment","fossil fuel"])
#my_keywords = set(['climate change', 'global warming', 'fossil fuel', 'methane', 'carbon', 'co2'])

Read in climate-related subreddits:

In [64]:
SUBREDDITS = pd.read_csv('CLIMATE_SUBREDDITS.txt',sep='\t',header=0)
SUBREDDITS.stance.value_counts()

pro     42
anti    20
neut    10
Name: stance, dtype: int64

In [65]:
SUBREDDITS.loc[SUBREDDITS.stance=='neut']

,subreddit,stance
4,science,neut
19,energy,neut
52,climatecmv,neut
65,Republican,neut
66,republicans,neut
67,askaconservative,neut
68,Conservative,neut
69,conservatives,neut
70,TrueConservativism,neut
71,AskTrumpSupporters,neut


In [66]:
len(set(SUBREDDITS['subreddit'])),len(SUBREDDITS['subreddit'])

(72, 72)

In [159]:
SUBREDDITS_LIST = list(SUBREDDITS['subreddit'])

# Create PRAW reddit instance to get posts and comments

In [4]:
reddit = praw.Reddit(client_id='1sbu376RCBiWRw',
                     client_secret='NbqiHMPiKicBXvgfrID-xVNktZM',
                     user_agent='mac:cc_framing:v1 (by /u/emma_cc_research)')

In [8]:
non_niche_subs = set(['AskReddit','environment','politics','worldnews','climateskeptics',
                     'Showerthoughts','climate','askscience','The_Donald','science','EcoInternet',
                     'collapse','explainlikeimfive','conspiracy','NoStupidQuestions','australia',
                     'unpopularopinion','climatechange','news','energy','canada','Conservative',
                     'skeptic','todayilearned','shittyaskscience','ChapoTrapHouse','CanadaPolitics',
                     'EverythingScience','worldpolitics','europe','AskScienceDiscussion',
                     'ClimateOffensive','changemyview','ClimateActionPlan','AskTrumpSupporters',
                     'GlobalWarming','GlobalClimateChange','esist','Green'])

In [5]:
p_id = 'aqnhy'
post = reddit.submission(id=p_id)

In [7]:
post.url

'http://www.sciencedaily.com/releases/2010/01/100114081543.htm'

In [8]:
post.is_self

False

In [9]:
post.selftext

'[deleted]'

In [9]:
n_members = {}
for sub_name in non_niche_subs:
    try:
        n_members[sub_name] = reddit.subreddit(sub_name).subscribers
    except NotFound:
        print('Not found:',sub_name)

Not found: ChapoTrapHouse
Not found: The_Donald


In [13]:
# According to https://people.com/politics/reddit-bans-the-donald-chapotraphouse-groups/
n_members['ChapoTrapHouse'] = 150000
n_members['The_Donald'] = 800000

In [14]:
pickle.dump(n_members,open('num_members_curated_subreddits.pkl','wb'))

In [132]:
# Can it know if a user is a bot?
vars(reddit.redditor("DragonFireDon"))

{'_reddit': <praw.reddit.Reddit at 0x11c21d630>,
 '_fetched': False,
 '_listing_use_sort': True,
 'name': 'DragonFireDon'}

In [16]:
reddit.redditor("AutoModerator").link_karma

13230

In [18]:
reddit.redditor("AutoModerator").__dict__

{'_reddit': <praw.reddit.Reddit at 0x1177ea240>,
 '_fetched': False,
 '_listing_use_sort': True,
 'name': 'AutoModerator'}

In [13]:
reddit.redditor("emma_cc_research")

Redditor(name='emma_cc_research')

## Get posts from r/spambotwatch

In [5]:
get_submissions(reddit,'spambotwatch')

Getting submissions and comments from: spambotwatch


In [ ]:
#get_submission_comments(reddit,'spambotwatch')

In [7]:
spambotwatch_df = pd.read_csv('praw_output/spambotwatch.tsv',sep='\t',header=0)
spambotwatch_df.title

0           JEEVAN BOBY (u/jeevanbobyvallickad) - Reddit
1                                 overview for funnynova
2      Spams Dating Tips Websites all with Stolen Con...
3                                 overview for poopcake5
4                                   overview for sutei_m
                             ...                        
190                               overview for royboy204
191                                 overview for mlg7732
192                            overview for 0wned_alover
193                                 overview for xrox333
194                                overview for frede933
Name: title, Length: 195, dtype: object

## Get posts from all subreddits

In [84]:
for i in range(len(SUBREDDITS['subreddit'])-1,len(SUBREDDITS['subreddit'])):
    SUBREDDIT = SUBREDDITS_LIST[i]
    get_submissions(reddit,SUBREDDIT)

Getting submissions and comments from: AskTrumpSupporters


## Inspect output: tsv of subreddits and meta

In [86]:
pd.read_csv('subreddits.tsv',sep='\t',header=None).drop_duplicates(0,keep='first')

,0,1,2
0,climateskeptics,Climate Skeptics: Trying to see through the al...,"Seeing past hyperbole, alarmism and environmen..."
1,skeptic,skeptic,## [Click this link to Read the Rules](http://...
2,climatechange,A place for a rational discussion on a divisiv...,This is a place for the rational discussion of...
3,climate,Information about the world's climate,Real and accurate data about the Earth's clima...
4,science,Reddit Science,# [Submission Rules](https://www.reddit.com/r/...
...,...,...,...
63,republicans,Republicans - RNC - GOP: Grand Old Party,"Republican, RNC and GOP news, issues, gossip, ..."
64,askaconservative,Ask A Conservative: Ask Conservatives And Repu...,#[Ask a Conservative](/r/askaconservative)\n\n...
65,Conservative,Conservative,#####\n**[Join us on discord.](https://discord...
66,conservatives,conservatives,"Conservatism (from, conservare, ""to preserve"")..."


## Inspect output: tsv of one subreddit's posts

In [88]:
df = pd.read_csv('praw_output/350.tsv',sep='\t',header=0)

In [89]:
df.columns

Index(['title', 'author', 'date', 'is_video', 'id', 'num_downs', 'num_ups',
       'upvote_ratio', 'num_comments', 'score', 'text', 'subreddit'],
      dtype='object')

In [90]:
df.is_video.value_counts()

False    127
Name: is_video, dtype: int64

In [91]:
df.subreddit.value_counts()

350    127
Name: subreddit, dtype: int64

In [160]:
#df.head(15)

## Get comments for all posts with non-zero num comments

In [156]:
for subreddit_tsv in glob.glob('praw_output/posts/*.tsv'):
    if os.path.exists('praw_output/post_comments/{}_COMMENTS.tsv'.format(subreddit_tsv.split('/')[-1][:-4])):
        print('Already got comments for subreddit {}'.format(subreddit_tsv))
    else:
        subreddit_posts = pd.read_csv(subreddit_tsv,sep='\t',header=0)
        if len(subreddit_posts) > 0:
            subreddit = str(subreddit_posts.iloc[0]['subreddit'])
            posts_with_comments = subreddit_posts.loc[subreddit_posts.num_comments > 0]
            if len(posts_with_comments) > 0:
                print('Getting comments from posts in subreddit: {}'.format(subreddit))
                for ix,row in posts_with_comments.iterrows():
                    get_submission_comments(reddit,subreddit,row['id'])
            else:
                print('0 comments among all posts in subreddit: {}'.format(subreddit))
        else:
            print('Subreddit {} has no posts'.format(subreddit_tsv))

Already got comments for subreddit praw_output/posts/350.tsv
Already got comments for subreddit praw_output/posts/350ppm.tsv
Already got comments for subreddit praw_output/posts/askaconservative.tsv
Already got comments for subreddit praw_output/posts/AskTrumpSupporters.tsv
Already got comments for subreddit praw_output/posts/carboncapture.tsv
Already got comments for subreddit praw_output/posts/carbontax.tsv
Already got comments for subreddit praw_output/posts/ccfunding.tsv
Already got comments for subreddit praw_output/posts/climate.tsv
Already got comments for subreddit praw_output/posts/climate_activism.tsv
Already got comments for subreddit praw_output/posts/climate_discussion.tsv
Already got comments for subreddit praw_output/posts/climate_science.tsv
Already got comments for subreddit praw_output/posts/ClimateActionPlan.tsv
Already got comments for subreddit praw_output/posts/climatechange.tsv
Already got comments for subreddit praw_output/posts/ClimateChangeCancer.tsv
Already g

# Use Pushshift API

In [18]:
multiple_search_term = '|'.join(KEYWORDS_LONG)
multiple_search_term

'global warming|climate change|carbon|fossil fuel|methane|environment|co2|climate crisis|climate emergency|extreme weather|2 degree|sustainable|clean energy|renewable|cap and trade|sea level rise|environmental justice|climate justice|COP|IPCC|deforestation|permafrost|glacier|drought|ecosystem|greenhouse gas|greenhouse effect|green new deal|EPA'

In [23]:
before_date = datetime.datetime.strptime("07-22-2020", "%m-%d-%Y")
after_date = datetime.datetime.strptime("07-20-2020", "%m-%d-%Y")
before_timestamp = int(datetime.datetime.timestamp(before_date))
after_timestamp = int(datetime.datetime.timestamp(after_date))

In [26]:
test_data = getPushshiftData(multiple_search_term,after_timestamp,before_timestamp,"comment")

In [61]:
test_data = getPushshiftData(multiple_search_term,after_timestamp,before_timestamp,"submission")

In [62]:
len(test_data)

0

In [67]:
# test_data[0
#         ]

In [32]:
def getPushshiftData(query, after, before, datatype):
    query_prefix = 'title' if datatype == 'submission' else 'q'
    url = 'https://api.pushshift.io/reddit/search/'+datatype+'/?'+\
    query_prefix+'='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)
    #print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']


def collectSubData(subm,subs_dict):
    try:
        title = subm['title']
    except KeyError:
        title = None
    try:
        url = subm['url']
    except KeyError:
        url = None
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"   
    try:
        author = subm['author']
    except KeyError:
        author = None
    sub_id = subm['id']
    try:
        score = subm['score']
    except KeyError:
        score = None
    try:
        created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    except KeyError:
        created = None
    try:
        numComms = subm['num_comments']
    except KeyError:
        numComms = None
    try:
        permalink = subm['permalink']
    except KeyError:
        permalink = None
    try:
        is_vid = subm['is_video']
    except KeyError:
        is_vid = None
    try:
        upvote_ratio = subm['upvote_ratio']
    except KeyError:
        upvote_ratio = None
    try:
        text = subm['selftext'].strip().replace('\t','').replace('\n','')
    except KeyError:
        text = ""
    try:
        subreddit = subm['subreddit']
    except KeyError:
        subreddit = None
    subData = {'id':sub_id,'title':title,'url':url,'author':author,'score':score,'date':created,
                    'num_comments':numComms,'permalink':permalink,'flair':flair,'is_video':is_vid,
                    'upvote_ratio':upvote_ratio,'text':text,'subreddit':subreddit}
    subs_dict[sub_id] = subData
    
    
def collectCommData(subm,subs_dict): 
    try:
        author = subm['author']
    except KeyError:
        author = None
    sub_id = subm['id']
    try:
        link_id = subm['link_id']
    except KeyError:
        link_id = None
    try:
        score = subm['score']
    except KeyError:
        score = None
    try:
        created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    except KeyError:
        created = None
    try:
        permalink = subm['permalink']
    except KeyError:
        permalink = None
    try:
        text = subm['body'].strip().replace('\t','').replace('\n','')
    except KeyError:
        text = ""
    try:
        subreddit = subm['subreddit']
    except KeyError:
        subreddit = None
    subData = {'id':sub_id,'link_id':link_id,'author':author,'score':score,'date':created,
                    'permalink':permalink,'text':text,'subreddit':subreddit}
    subs_dict[sub_id] = subData

In [21]:
def pushshift_wrapper(after_str, before_str, datatype, query=None, keywords=None):
    failed_requests = []
    
    if query is None:
        query = '|'.join(keywords)
    
    subCount = 0
    subStats = {}
    before_date = datetime.datetime.strptime(before_str, "%m-%d-%Y")
    after_date = datetime.datetime.strptime(after_str, "%m-%d-%Y")
    before_timestamp = int(datetime.datetime.timestamp(before_date))
    after_timestamp = int(datetime.datetime.timestamp(after_date))
    print("Getting all submissions with query '{}' from {} to {}".format(query,
                                                                         after_str,before_str))
    try:
        data = getPushshiftData(query, after_timestamp, before_timestamp, datatype)
        # Will run until all posts have been gathered 
        # from the 'after' date up until before date
        while len(data) > 0:
            for submission in data:
                if datatype == "submission":
                    collectSubData(submission,subStats)
                else:
                    collectCommData(submission,subStats)
                subCount+=1
            # Calls getPushshiftData() with the created date of the last submission
            #print(len(data))
            #print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
            after_timestamp = data[-1]['created_utc']
            #print(after_timestamp)
            try:
                data = getPushshiftData(query, after_timestamp, before_timestamp, datatype)
            except JSONDecodeError:
                failed_requests.append((query,after_timestamp,before_timestamp,datatype))

        print('Num submissions:',subCount,len(subStats))

        interim_df = pd.DataFrame(list(subStats.values()))
        #print(interim_df)

        datatype_prefix = 'posts' if datatype == 'submission' else 'post_comments'
        out_dir = os.path.join('pushshift_output',datatype_prefix,'{}_to_{}'.format(after_str,before_str))
        failed_reqs_out_dir = os.path.join('pushshift_output','failed_requests',
                                                      '{}_{}'.format(after_str,before_str))
        if not os.path.exists(out_dir):
            os.mkdir(out_dir)
        if not os.path.exists(failed_reqs_out_dir):
            os.mkdir(failed_reqs_out_dir)
            
        if keywords is None:
            interim_df.to_pickle(os.path.join(out_dir,'{}.pkl'.format(query)))
            print('Saved query submissions to {}!'.format(os.path.join(out_dir,'{}.pkl'.format(query))))
            pickle.dump(failed_requests,open(os.path.join(failed_reqs_out_dir,'{}.pkl'.format(query)),'wb'))
        else:
            interim_df.to_pickle(os.path.join(out_dir,'{}.pkl'.format('keywords_long')))
            print('Saved query submissions to {}!'.format(os.path.join(out_dir,'{}.pkl'.format('keywords_long'))))
            pickle.dump(failed_requests,open(os.path.join(failed_reqs_out_dir,'{}.pkl'.format('keywords_long')),'wb'))
            
    except JSONDecodeError:
        failed_requests.append((query,after_timestamp,before_timestamp,datatype))
        print("First request failed")

In [25]:
#keywords_missing = []

In [30]:
# for start_year in range(2010,2015,1):
#     end_year = start_year+1
#     for keyword in KEYWORDS_LONG:
#         if not os.path.exists(os.path.join('pushshift_output','posts',
#                                            '1-1-{}_to_12-31-{}'.format(start_year,start_year),
#                                            '{}.pkl'.format(keyword))):
#             print("Missing {}, {}".format(keyword,start_year))
#             keywords_missing.append(keyword)
#             #pushshift_wrapper('1-1-{}'.format(start_year),'12-31-{}'.format(start_year),'submission',
#             #query=None,keywords=keywords_missing)
# print('************')    
# for start_year in range(2010,2015,1):
#     end_year = start_year+1
#     for keyword in KEYWORDS_LONG:
#         if not os.path.exists(os.path.join('pushshift_output','post_comments',
#                                            '1-1-{}_to_12-31-{}'.format(start_year,start_year),
#                                            '{}.pkl'.format(keyword))):
#             print("Missing {}, {}".format(keyword,start_year))
#             #pushshift_wrapper(keyword,'1-1-{}'.format(start_year),'12-31-{}'.format(start_year),'comment')

In [31]:
# for start_year in range(2015,2021,1):
#     end_year = start_year+1
#     pushshift_wrapper('1-1-{}'.format(start_year),'12-31-{}'.format(start_year),'submission',
#                       query=None,keywords=keywords_missing)

# Get all comments attached to a post

## PRAW

### Get IDs

In [410]:
posts_with_comments_ids = pickle.load(open('output/posts_with_comments_ids.pkl','rb'))
print(len(posts_with_comments_ids))
sub_ids_to_fetch = list(posts_with_comments_ids)

419100

In [420]:
# with open('output/comment_ids_per_post.tsv','w') as f:
#     f.write("{}\t{}\n".format('post_id','comment_ids'))

In [439]:
def praw_get_comments(sub_id):
    try:
        post = reddit.submission(id=sub_id)
        post_author = post.author
        post_title = post.title
        post_comms = list(post.__dict__['_comments_by_id'].keys())
        #print(len(post_comms))
        comments_per_post[sub_id] = post_comms

        with open('output/comment_ids_per_post.tsv','a') as f:
            f.write("{}\t{}\n".format(sub_id,','.join(post_comms)))
    except Forbidden:
        pass

In [461]:
for ix_sub_id in range(174623,len(sub_ids_to_fetch)):
    sub_id = sub_ids_to_fetch[ix_sub_id]
    praw_get_comments(sub_id)
    
    if ix_sub_id % 1000 == 0:
        print(ix_sub_id)

175000
176000
177000
178000
179000
180000
181000
182000
183000
184000
185000
186000
187000
188000
189000
190000
191000
192000
193000
194000
195000
196000
197000
198000
199000
200000
201000
202000
203000
204000
205000
206000
207000
208000
209000
210000
211000
212000
213000
214000
215000
216000
217000
218000
219000
220000
221000
222000
223000
224000
225000
226000
227000
228000
229000
230000
231000
232000
233000
234000
235000
236000
237000
238000
239000
240000
241000
242000
243000
244000
245000
246000
247000
248000
249000
250000
251000
252000
253000
254000
255000
256000
257000
258000
259000
260000
261000
262000
263000
264000
265000
266000
267000
268000
269000
270000
271000
272000
273000
274000
275000
276000
277000
278000
279000
280000
281000
282000
283000
284000
285000
286000
287000
288000
289000
290000
291000
292000
293000
294000
295000
296000
297000
298000
299000
300000
301000
302000
303000
304000
305000
306000
307000
308000
309000
310000
311000
312000
313000
314000
315000
316000
317000

In [460]:
ix_sub_id,sub_id

(174623, '92efdf')

In [447]:
len(comments_per_post)

11720

In [462]:
pd.read_csv('output/comment_ids_per_post.tsv',sep='\t')

,post_id,comment_ids
0,anbg0,"t1_c0ignhg,t1_c0ignz0,t1_c0igq3n"
1,apn8b,"t1_c0irhej,t1_c0is8sw"
2,aprgq,"t1_c0isfnb,t1_c0is90g,t1_c0iscd2,t1_c0isbol,t1..."
3,aprgv,"t1_c0is3y4,t1_c0ism1k,t1_c0isyft,t1_c0isvkx,t1..."
4,aprie,"t1_c0is4by,t1_c0islml,t1_c0ismfv,t1_c0isk7q,t1..."
...,...,...
403412,3yuldj,"t1_cygrqj0,t1_cygrhs5,t1_cygtcrx,t1_cygtrvf,t1..."
403413,3yulsl,"t1_cylxj3h,t1_cygqso3"
403414,3yup0j,"t1_cyhjhnw,t1_cylxiyc"
403415,3yuytd,"t1_cygu9uo,t1_cygx6g5,t1_cyhpqys,t1_cygxgig,t1..."


### Get text

In [405]:
comment = reddit.comment("c0st842")
comment.body

'Personally I am more concerned about the fact that methane is an extremely effective greenhouse gas.  '

In [ ]:
comment

In [ ]:
comment_ids_per_post = pd.read_csv('output/comment_ids_per_post.tsv',sep='\t')
comment_ids = comment_ids_per_post['comment_ids']
all_comment_ids = [x.split(',') for x in comment_ids]
all_comment_ids = [item for sublist in all_comment_ids for item in sublist]
unique_comment_ids = set(all_comment_ids)

In [ ]:
print(len(all_comment_ids),len(unique_comment_ids))

In [ ]:
# Dataframe of 
# comment_id | text

with open('output/text_per_comment.tsv','w') as f:
    f.write('{}\t{}\n'.format('comment_id','text'))
    
unique_comment_ids = list(unique_comment_ids)

In [ ]:
for c_id_index in range(len(unique_comment_ids)):
    c_id = unique_comment_ids[c_id_index]
    comm = reddit.comment(c_id)
    comment_body = strip_whitespace(comm.body)
    
    with open('output/text_per_comment.tsv','a') as f:
        f.write('{}\t{}\n'.format(c_id,comment_body))

## Pushshift -- doesn't seem to work >:(

In [290]:
def getComments(sub_id):
    url = 'https://api.pushshift.io/reddit/submission/comment_ids/{}'.format(sub_id)
    r = requests.get(url)
    sub_comm_data = json.loads(r.text)
    return sub_comm_data['data']

In [45]:
def pushshift_comment_wrapper(link_id, after_str, before_str, datatype):
    failed_requests = []
    
    subCount = 0
    subStats = {}
    before_date = datetime.datetime.strptime(before_str, "%m-%d-%Y")
    after_date = datetime.datetime.strptime(after_str, "%m-%d-%Y")
    before_timestamp = int(datetime.datetime.timestamp(before_date))
    after_timestamp = int(datetime.datetime.timestamp(after_date))
    print("Getting all comments with link_id '{}' from {} to {}".format(link_id,after_str,before_str))
    try:
        data = getComments(link_id, after_timestamp, before_timestamp, datatype)
        # Will run until all posts have been gathered 
        # from the 'after' date up until before date
        while len(data) > 0:
            for submission in data:
                if datatype == "submission":
                    collectSubData(submission,subStats)
                else:
                    collectCommData(submission,subStats)
                subCount+=1
            # Calls getPushshiftData() with the created date of the last submission
            #print(len(data))
            #print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
            after_timestamp = data[-1]['created_utc']
            #print(after_timestamp)
            try:
                data = getComments(link_id, after_timestamp, before_timestamp, datatype)
            except JSONDecodeError:
                failed_requests.append((link_id,after_timestamp,before_timestamp,datatype))

        print('Num submissions:',subCount,len(subStats))

        interim_df = pd.DataFrame(list(subStats.values()))
        #print(interim_df)

        datatype_prefix = 'posts' if datatype == 'submission' else 'linked_post_comments'
        out_dir = os.path.join('output','pushshift_output',datatype_prefix,'{}_to_{}'.format(after_str,before_str))
        failed_reqs_out_dir = os.path.join('pushshift_output','failed_requests',
                                                      '{}_{}'.format(after_str,before_str))
        if not os.path.exists(out_dir):
            os.mkdir(out_dir)
        if not os.path.exists(failed_reqs_out_dir):
            os.mkdir(failed_reqs_out_dir)
            
        interim_df.to_pickle(os.path.join(out_dir,'{}.pkl'.format(link_id)))
        print('Saved query submissions to {}!'.format(os.path.join(out_dir,'{}.pkl'.format(link_id))))
        pickle.dump(failed_requests,open(os.path.join(failed_reqs_out_dir,'{}.pkl'.format(link_id)),'wb'))
            
    except JSONDecodeError:
        failed_requests.append((query,after_timestamp,before_timestamp,datatype))
        print("First request failed")

In [390]:
getComments(sub_id)

[]

In [391]:
url = 'https://api.pushshift.io/reddit/submission/search/?ids={}&limit=1000'.format(sub_id)
r = requests.get(url)
sub_data = json.loads(r.text)

In [392]:
len(sub_data['data'])

1

In [ ]:
#sub_data['data'][0]

In [394]:
url = 'https://api.pushshift.io/reddit/comment/search/?id=43u5cw'
r = requests.get(url)
comm_data = json.loads(r.text)

In [ ]:
#comm_data

In [395]:
url = 'https://api.pushshift.io/reddit/submission/comment_ids/{}'.format(sub_id)
r = requests.get(url)
sub_comm_data = json.loads(r.text)

In [396]:
sub_comm_data['data']

[]

In [397]:
url = 'https://api.pushshift.io/reddit/comment/search/?link_id={}'.format(sub_id)
r = requests.get(url)
comm_from_sub_data = json.loads(r.text)

In [398]:
comm_from_sub_data['data']

[]

In [301]:
comm_ids = [x['id'] for x in comm_from_sub_data['data']]
comm_ids

[]

In [286]:
set(comm_ids).issubset(set(sub_comm_data['data']))

True

In [287]:
len(comm_ids),len(sub_comm_data['data'])

(25, 71)